In [ ]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Initialize LLM with steering vector capability
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)

In [ ]:
import json
file_path = "/home/xhl/eval/my_eval/data/math/train.jsonl"

problems = []
answers = []

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        problems.append(item["problem"])
        answers.append(item["answer"])
examples = ["Please reason step by step, and put your final answer within \\boxed{}.\nUser: " + prompt + "\nAssistant: <think>" for prompt in problems]

In [ ]:
import time
total_output_tokens=0
times = 0
for i in range(10):
    start_time = time.time()
    example_answers = llm.generate(
        examples[i],
        SamplingParams(
            temperature=0,
            max_tokens=2048,
            skip_special_tokens=False,
        ),
    )
    end_time = time.time()
    times+=end_time-start_time
    total_output_tokens += sum((len(r.outputs[0].token_ids) if r.outputs else 0) for r in example_answers)

toks_per_sec = total_output_tokens / (end_time - start_time) if end_time - start_time > 0 else 0
print(f"{(times)/10:.8f}s/req")
print(f"output toks/s: {toks_per_sec:.2f}")

# # batch
# example_answers = llm.generate(
#     examples[:512],
#     SamplingParams(
#         temperature=0,
#         max_tokens=2048,
#         skip_special_tokens=False,
#     ),
# )

